In [ ]:
# Questions left to answer:

# How to deal with PTVs that have the same prescription dose - what should be that structure's name?

# For composite doses, do we want to allow higher dose constraints to remain in the CSV/XML file?

In [1]:
# ============================= GUI CREATION ===============================
filepath='//kodiak/physics/lpadilla/Clinic/10 week sprint - Peer review/Finished Products/DC sheets/'

import numpy as np
import pandas as pd
import re
import XMLconversion 

# GUI libraries
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog


class GUIwindow(Frame):

    def fileselect(self):
        
        self.fullfilename = filedialog.askopenfilename(initialdir = filepath,
                                                   title = "Select Template",
                                                   filetypes = (("CSV files","*.csv"),("all files","*.*")))
        
        self.filename = self.fullfilename.replace(filepath,"")
        
        self.label.config(text=self.filename)
        
        # extract the information needed to adjust user prompts 
        templatename=self.filename[:-6]
        self.TemplateType, self.TxRegionFxType, self.TxSite, DoseInfo=templatename.split(' - ')
        _,numfractions = DoseInfo.split('cGy')
        
        # if the number of fractions is embedded in the template name, make it available to the gui
        if len(numfractions) is not 9:
            self.numfractions= IntVar()
            self.numfractions=numfractions
   

    def PTVnumPrompt(self):
        
        #  Check to see if one of the radio buttons has already been pressed, if so, clear all previous info
        if hasattr(self,'PTVnum'):
            self.PTVnumLabel.destroy()
            self.PTVnumEntry.destroy()
            self.DoseInfoButton.destroy()
            self.clear()
            
       
        # Create the prompt for the user to insert the number of PTVs for that treatment
        self.PTVnumLabel = Label(root,text='Indicate total number of PTVs: ',pady=5)
        self.PTVnumLabel.grid(row=8, column=0, sticky=E)
    
        self.PTVnum = IntVar()
        self.PTVnumEntry = Entry(root, textvariable=self.PTVnum,bd=3)
        self.PTVnumEntry.grid(row=8, column=1, sticky=W)
        
        # Create the button that will display dose information to be filled out according to user input
        self.DoseInfoButton = Button(root, text="Enter PTV dose info",command=self.DoseInfo)
        self.DoseInfoButton.grid(row=8,column=2, sticky=E)
   
    
    def DoseInfo(self):
            
        # label the dose table.  No need for PTV labels since these will be set by row
        self.DoseColLabel = Label(root,text='Dose in cGy:')
        self.DoseColLabel.grid(row=9,column=1)
        self.FractionsColLabel = Label(root,text='Number of Fractions:')
        self.FractionsColLabel.grid(row=9,column=2)
        
                  
        # For no boost, the number of fractions will be the same for all PTVs
        if self.booststatus.get() is 1:
            
            self.PTVDose = IntVar()

            # Create all the entry boxes for the different PTV doses 
            for i in range(0,self.PTVnum.get(),1):
                self.PTVRowLabel=Label(root,text='PTV '+ str(i+1))         
                self.PTVRowLabel.grid(row=i+10,column=0)   
                self.PTVDoseEntry=Entry(root, textvariable=self.PTVDose)
                self.PTVDoseEntry.grid(row=i+10,column=1) 
                    
            # Check to see if the number of fractions was given by the template, if so, just transfer the numfractions
            if hasattr(self,'numfractions') and self.flag==0:
                self.FractionInfo=Label(root, text=self.numfractions).grid(row=9, column=2)
            else:
                self.numfractions = IntVar()
                self.FractionInfo=Entry(root, textvariable=self.numfractions)
                self.FractionInfo.grid(row=10,column=2)
        
        
        # Create the number of fractions entries based on the boost status
        
        # For SIB, the number of fractions will be the same for all PTVs
        if self.booststatus.get() is 2:
            
            # declare the PTVDose list and give it the right size based on the number of PTVs indicated
            self.PTVDose = []
            i=0
            
            for i in range(0,self.PTVnum.get(),1):
                self.PTVDose.append(IntVar())

                # Create all the entry boxes for the different PTV doses 
            for i in range(0,self.PTVnum.get(),1):
                self.PTVRowLabel=Label(root,text='PTV '+ str(i+1))         
                self.PTVRowLabel.grid(row=i+10,column=0)   
                self.PTVDoseEntry=Entry(root, textvariable=self.PTVDose[i])
                self.PTVDoseEntry.grid(row=i+10,column=1) 
            
            # Check to see if the number of fractions was given by the template, if so, just transfer the numfractions
            if hasattr(self,'numfractions') and self.flag==0:
                self.FractionInfo=Label(root, text=self.numfractions).grid(row=10, column=2)
            else:
                self.numfractions = IntVar()
                self.FractionInfo=Entry(root, textvariable=self.numfractions)
                self.FractionInfo.grid(row=10,column=2)
        
        # For Sequential boost, the number of fractions is given per PTV
        elif self.booststatus.get() is 3:
            
            # declare the PTVDose list and give it the right size based on the number of PTVs indicated
            self.PTVDose = []
            i=0
            
            for i in range(0,self.PTVnum.get(),1):
                self.PTVDose.append(IntVar())

           # Create all the entry boxes for the different PTV doses 
            for i in range(0,self.PTVnum.get(),1):
                self.PTVRowLabel=Label(root,text='PTV '+ str(i+1))         
                self.PTVRowLabel.grid(row=i+10,column=0)   
                self.PTVDoseEntry=Entry(root, textvariable=self.PTVDose[i])
                self.PTVDoseEntry.grid(row=i+10,column=1) 
            
            # declare the numfractions list and give it the right size based on the number of PTVs indicated
            self.numfractions = []
           
            for i in range(0,self.PTVnum.get(),1):
                self.numfractions.append(IntVar())            
           
                
           # Create all the entry boxes for the different number of fractions
            for i in range(0,self.PTVnum.get(),1):
                self.FractionInfo=Entry(root, textvariable=self.numfractions[i])
                self.FractionInfo.grid(row=i+10,column=2)   
                
        self.CreateSheetButton = Button(root,text = 'Create Dose Constraint Sheet', command=self.createFiles)
#         self.CreateSheetButton = Button(root,text = 'Create Dose Constraint Sheet')
        self.CreateSheetButton.grid(row=self.PTVnum.get()+10,sticky=E, pady=3, columnspan=3)
        
        self.ClearDoseInfo = Button(root, text = 'Clear Dose Information', command=self.clear)
        self.ClearDoseInfo.grid(row=self.PTVnum.get()+10, sticky=W, columnspan=3)
        
        # print messages as files are created
        self.CompletionCSV = Label(root)
#         self.CompletionCSV.config(text = 'Enter the dose fractionation information above and press' + 
#                                   '"Create Dose Constraint Sheet".', pady = 3)
        self.CompletionCSV.grid(row=self.PTVnum.get()+11, columnspan=3)
        self.CompletionXML = Label(root)
#         self.CompletionXML.config(text = 'The program will notify you when the CSV and XML files have been created.', pady = 3)
        self.CompletionXML.grid(row=self.PTVnum.get()+12, columnspan=3) 
          
        
        # this flag serves to ensure that all the previous dose information is cleared if the user presses ClearDoseInfo 
        self.flag=self.flag+1

    def clear(self):
        
        for child in root.grid_slaves():
            grid_info = child.grid_info()

            if grid_info['row'] > 9 :
                child.destroy()

    def createFiles(self):
        
        return XMLconversion.createFiles(self), self
        
    def createWidgets(self):
        
        # Display the VCU Massey Cancer Center logo at the top of the GUI window
        self.logoImg = PhotoImage(file = '//kodiak/physics/lpadilla/Clinic/10 week sprint - Peer review/massey_hz_gold.png')
#         self.logoImg = logoOriginal.subsample(3,3)
        self.photo = Label(root, image= self.logoImg, bd=6, width = 600)
        self.photo.grid(row=0,columnspan=3)
        
        # GUI item:  select the template to use as the basis of the patient-specific CSV file
        self.button = Button(root,text="Select Template",command=self.fileselect)
        self.button.grid(row=1,columnspan=3)
    
        # display the name of the template selected to confirm
        self.label = Label(root)  
        self.label.grid(row=2,columnspan=3)
        
        # GUI item:  selection option for boost in treatment (no, SIB, sequential)
        # title of this item
        self.boostText = StringVar()
        self.boostText.set('Does this treatment have a boost?')
       
        self.boostLabel = Label(root,textvariable=self.boostText,padx=4, pady=4)
        self.boostLabel.grid(row=4,sticky=W,column=0)
    
        # radiobuttons for each option
        self.booststatus = IntVar()
        self.R1 = Radiobutton(root, text="Option 1:  No Boost", variable=self.booststatus, value=1, 
                              command=self.PTVnumPrompt)
        self.R1.grid(row=5, sticky=W,column=0)

        self.R2 = Radiobutton(root, text="Option 2:  SIB", variable=self.booststatus, value=2, 
                              command=self.PTVnumPrompt)
        self.R2.grid(row=6, sticky=W,column=0)
    
        self.R3 = Radiobutton(root, text="Option 3:  Sequential Boost", variable=self.booststatus, value=3, 
                              command=self.PTVnumPrompt)
        self.R3.grid(row=7, sticky=W,column=0)        

        self.PriorRTstatus = IntVar()
        self.PriorRT = Checkbutton(root, text="Evaluation of Dose Composite with Prior RT", variable=self.PriorRTstatus, 
                                   onvalue=1, offvalue=0)
        self.PriorRT.grid(row=4, sticky=W,column=2)   
        
        
    def __init__(self, master=None):
        
        Frame.__init__(self, master)
        self.filename = None
        # this flag serves to ensure that all the previous dose information is cleared if the user presses ClearDoseInfo 
        self.flag = 0
        self.createWidgets()


root = Tk()
root.title("Treatment information")
app = GUIwindow(master=root)
app.mainloop()





In [ ]:
# CODE TO DEVELOP ADDITIONAL FEATURES FOR THE GUI


self= app
# Use GUI info and template info to create the dose constraints sheet
objectives = pd.read_csv(self.fullfilename)
finalfile = self.TxSite + ' (' 
doseprescription = pd.DataFrame({'PTVdose':np.empty(self.PTVnum.get()),'numfx':np.empty(self.PTVnum.get()),
                                 'fxdose':np.empty(self.PTVnum.get())})

 # for no boost
if self.booststatus.get() is 1:

    # assign the number of fractions entered by the user either from selecting a template with embedded 
    # number of fractions in it    
    if type(self.numfractions) is str:
        numfx=int(self.numfractions.get())

    # or by extracting the typed number of fractions by the user
    else:
        numfx=self.numfractions.get()       

    # create a dataframe to hold the dose prescription information entered by the user
    for i in range(0,self.PTVnum.get(),1):
        doseprescription.PTVdose[i]=self.PTVDose.get()
        doseprescription.numfx[i]=numfx
        doseprescription.fxdose[i]=doseprescription.PTVdose[i]/doseprescription.numfx[i]

    # sort the dose prescription dataframe from lowest to highest dose
    doseprescription.sort_values('PTVdose', axis=0, ascending=True, inplace=True)
    doseprescription=doseprescription.reset_index(drop=True) 

    # finish creating the final file name
    finalfile+=str(int(doseprescription.fxdose[0]))+'cGy x ' +  str(int(doseprescription.numfx[0])) + 'fx)'  

# for SIB
if self.booststatus.get() is 2: 

    # assign the number of fractions entered by the user either from selecting a template with embedded 
    # number of fractions in it
    if type(self.numfractions) is str:
        numfx=int(self.numfractions.get())

    # or by extracting the typed number of fractions by the user    
    else:
        numfx=self.numfractions.get()

    # create a dataframe to hold the dose prescription information entered by the user    
    for i in range(0,self.PTVnum.get(),1):
        doseprescription.PTVdose[i]=self.PTVDose[i].get()
        doseprescription.numfx[i]=numfx
        doseprescription.fxdose[i]=doseprescription.PTVdose[i]/doseprescription.numfx[i]

    # sort the dose prescription dataframe from lowest to highest dose    
    doseprescription.sort_values('PTVdose', axis=0, ascending=True, inplace=True)
    doseprescription=doseprescription.reset_index(drop=True)  

   # finish creating the final file name
    for i in range(0,self.PTVnum.get()-1,1):
        finalfile+=str(int(doseprescription.fxdose[i]))+'-'

    finalfile+= str(int(doseprescription['fxdose'].iloc[-1]))+'cGy x ' +  str(int(doseprescription['numfx'].iloc[-1])) + 'fx)'

# for sequential boost    
if self.booststatus.get() is 3:

    # create a dataframe to hold the dose prescription information entered by the user    
    for i in range(0,self.PTVnum.get(),1):
        doseprescription.PTVdose[i]=self.PTVDose[i].get()
        doseprescription.numfx[i]=self.numfractions[i].get()
        doseprescription.fxdose[i]=doseprescription.PTVdose[i]/doseprescription.numfx[i]

    # sort the dose prescription dataframe from lowest to highest dose    
    doseprescription.sort_values('PTVdose', axis=0, ascending=True, inplace=True)
    doseprescription=doseprescription.reset_index(drop=True)

    # finish creating the final file name    
    finalfile+=str(int(doseprescription.fxdose[0]))+'cGy x ' +  str(int(doseprescription.numfx[0])) + '-'

    for i in range(1,self.PTVnum.get()-1,1):
        finalfile+= str(int(doseprescription.numfx[i])) + '-'


    finalfile+= str(int(doseprescription['numfx'].iloc[-1])) + 'fx)'  


# Edit the objectives loaded from the template to reflect the dose fractionation inserted by the user

# Start by replacing the "_DoseincGy" by the prescription dose in the PTV coverage constraints
PTVcoverage = pd.DataFrame(objectives.iloc[0,:]).transpose()
PTVcoverage.iloc[0,0]=PTVcoverage.iloc[0,0].replace('DoseincGy',str(int(doseprescription.PTVdose[0])) )
PTVcoverage.iloc[0,3]=PTVcoverage.iloc[0,3].replace('Dose',str(round(float((doseprescription.PTVdose[0])/100),1)) )

if self.PTVnum.get()>1:

    for i in range(1,self.PTVnum.get(),1):
        PTVcoverage = PTVcoverage.append(pd.DataFrame(objectives.iloc[0,:]).transpose())
        PTVcoverage.iloc[i,0]=PTVcoverage.iloc[i,0].replace('DoseincGy',str(int(doseprescription.PTVdose[i])) )
        PTVcoverage.iloc[i,3]=PTVcoverage.iloc[i,3].replace('Dose',str(round(float((doseprescription.PTVdose[i])/100),1)))

# Now, tailor the D0.03cc to the user entry 
# copy the entry for D0.03cc
PTVmax = pd.DataFrame(objectives.iloc[1,:]).transpose()

# split the "Evaluator" entry into '<' and the multiplicative factor, e.g. '1.1'
temp = re.split('Dosex',PTVmax.iloc[0,4]) 

# calculate the max dose for the PTV with the highest prescription
maxdose = round(float(temp[1])*(doseprescription.PTVdose[self.PTVnum.get()-1])/100,1)

# create the evaluator with this information
PTVmax.iloc[0,4] = '<'+str(maxdose)

# if there is an acceptable variation for Dmax, also insert the dose
if type(PTVmax.iloc[0,5]) == str:
    # split the "Evaluator" entry into '<' and the multiplicative factor, e.g. '1.1'
    tempav = re.split('Dosex',PTVmax.iloc[0,5]) 

    # calculate the max dose for the PTV with the highest prescription
    maxdoseav = round(float(tempav[1])*(doseprescription.PTVdose[self.PTVnum.get()-1])/100,1)

    # create the evaluator with this information
    PTVmax.iloc[0,5] = '<'+str(maxdoseav)

# replace the Structure ID name with the correct name
PTVmax.iloc[0,0] = PTVcoverage.iloc[self.PTVnum.get()-1,0]


# create the dose evaluation parameters for the PTV using coverage and D0.03cc
PTVdoseeval = PTVcoverage.append(PTVmax)
PTVdoseeval = PTVdoseeval.reset_index(drop=True) 

if (self.booststatus.get() is 1) and (self.PTVnum.get()>1):
    for i in range(0,self.PTVnum.get()-1,1):
        # create the dose evaluation parameters for the PTV using coverage and D0.03cc
        PTVdoseeval = PTVdoseeval.append(PTVmax)
        PTVdoseeval = PTVdoseeval.reset_index(drop=True)         

# update the objectives dataframe
objectives = PTVdoseeval.append(objectives.iloc[2:,:])
objectives = objectives.reset_index(drop=True)        




In [ ]:
PTVdoseeval

In [ ]:
type(PTVmax.iloc[0,5]) == str

In [ ]:

self= app
# Use GUI info and template info to create the dose constraints sheet
objectives = pd.read_csv(self.fullfilename)
finalfile = self.TxSite + ' (' 
doseprescription = pd.DataFrame({'PTVdose':np.empty(self.PTVnum.get()),'numfx':np.empty(self.PTVnum.get()),
                                 'fxdose':np.empty(self.PTVnum.get())})

 # for no boost
if self.booststatus.get() is 1:

    # assign the number of fractions entered by the user either from selecting a template with embedded 
    # number of fractions in it    
    if type(self.numfractions) is str:
        numfx=int(self.numfractions.get())

    # or by extracting the typed number of fractions by the user
    else:
        numfx=self.numfractions.get()       

    # create a dataframe to hold the dose prescription information entered by the user
    for i in range(0,self.PTVnum.get(),1):
        doseprescription.PTVdose[i]=self.PTVDose.get()
        doseprescription.numfx[i]=numfx
        doseprescription.fxdose[i]=doseprescription.PTVdose[i]/doseprescription.numfx[i]

    # sort the dose prescription dataframe from lowest to highest dose
    doseprescription.sort_values('PTVdose', axis=0, ascending=True, inplace=True)
    doseprescription=doseprescription.reset_index(drop=True) 

    # finish creating the final file name
    finalfile+=str(int(doseprescription.fxdose[0]))+'cGy x ' +  str(int(doseprescription.numfx[0])) + 'fx)'  

In [ ]:
doseprescription

In [ ]:
# if there is an acceptable variation for Dmax, also insert the dose
if PTVmax.iloc[0,5] is not 'NaN':
    # split the "Evaluator" entry into '<' and the multiplicative factor, e.g. '1.1'
    tempav = re.split('Dosex',PTVmax.iloc[0,5]) 

    # calculate the max dose for the PTV with the highest prescription
    maxdoseav = round(float(tempav[1])*(doseprescription.PTVdose[self.PTVnum.get()-1])/100,1)

    # create the evaluator with this information
    PTVmax.iloc[0,5] = '<'+str(maxdoseav)

# replace the Structure ID name with the correct name
PTVmax.iloc[0,0] = PTVcoverage.iloc[self.PTVnum.get()-1,0]

# create the dose evaluation parameters for the PTV using coverage and D0.03cc
PTVdoseeval = PTVcoverage.append(PTVmax)
PTVdoseeval = PTVdoseeval.reset_index(drop=True) 

# update the objectives dataframe
objectives = PTVdoseeval.append(objectives.iloc[2:,:])
objectives = objectives.reset_index(drop=True) 

In [ ]:
objectives

In [ ]:
#___________________________________CONVERT DOSE CONSTRAINTS CSV FILE TO PLAN OBJECTIVES XML___________________________________

#*********** Create the dataframe with all the dose constraint info to pass on to the XML file ************

# XMLdata is that data frame that will contain all the dose constraint information necessary to convert the CSV dose constraints
# file into and XML

#Create the empty dataframe
XMLdata= pd.DataFrame(0,index=range(0,len(objectives)),columns=['ObjType','I1','I2','I3','I4','I5','I6'])

# Give it the structures that each objective refers to.  This comes from the "objectives" dataframe that reflects the info
# in the CSV datafile 
XMLdata.I1=objectives['Structure ID']

#figure out the indeces of the objectives that are "bottom" type in eclipse clinical protocol's plan objectives
Vobj= objectives['DVH Objective'].str.startswith('V')
Dobj=objectives['DVH Objective'].str.contains('D') 
GIobj= objectives['DVH Objective'].str.startswith('GI')
CIobj= objectives['DVH Objective'].str.startswith('CI')
bottom=Vobj|Dobj|GIobj|CIobj

#Tag each objective with the "top" or "bottom" classification so they can be properly formated in the XML file
XMLdata.ObjType[bottom]='bottom'
XMLdata.ObjType[~bottom]='top'


#Translate each objective into its "Type" value according to the 'dictionary for plan objectives', and set the XMLdata 
# value accordingly
#do not have to worry about setting the "0" types because 0 is the default value of the XMLdata matrix

Type1=GIobj
Type2=objectives['DVH Objective'].str.startswith('V') & ~objectives['DVH Objective'].str.contains('Gy')
Type3=objectives['DVH Objective'].str.startswith('V') & objectives['DVH Objective'].str.contains('Gy')
Type4=objectives['DVH Objective'].str.startswith('D') & ~objectives['DVH Objective'].str.contains('cc')
Type5=objectives['DVH Objective'].str.startswith('D') & objectives['DVH Objective'].str.contains('cc')


XMLdata.I2[Type1]=1
XMLdata.I2[Type2]=2
XMLdata.I2[Type3]=3
XMLdata.I2[Type4]=4
XMLdata.I2[Type5]=5

# Translate evaluators into modifier values.  Again, for more specifics on what each modifier value means, refer to the 
# 'dictionary for plan objectives' file
#the default XMLdata value is 0, so we don't have to worry about setting 0 values.
# mod5 and mod6 are not defined because mod5 is never used in our clinic, and mod6 has no definition we could find based on
# the approach we took to develop the dictionary for plan objectives

Mod1top=objectives['Evaluator'].str.startswith('<') & XMLdata.ObjType.isin(['top']) 
Mod2top=objectives['DVH Objective'].str.contains('mean', case=False) & objectives['Evaluator'].str.startswith('=') & XMLdata.ObjType.isin(['top']) 
Mod3top=objectives['DVH Objective'].str.contains('max', case=False) & objectives['Evaluator'].str.startswith('=')  & XMLdata.ObjType.isin(['top'])
Mod4top=objectives['DVH Objective'].str.contains('min', case=False) & objectives['Evaluator'].str.startswith('=')  & XMLdata.ObjType.isin(['top']) 
#Mod5top=objectives['DVH Objective'].str.contains('max', case=False) & objectives['Evaluator'].str.startswith('=')  & XMLdata.ObjType.isin(['top']) 
#Mod6top we didn't find any objectives with a modifier value of 6 when creating our dictionary
Mod7top=objectives['DVH Objective'].str.contains('mean', case=False) & objectives['Evaluator'].str.startswith('>')  & XMLdata.ObjType.isin(['top']) 
Mod8top=objectives['DVH Objective'].str.contains('mean', case=False) & objectives['Evaluator'].str.startswith('<')  & XMLdata.ObjType.isin(['top']) 
Mod9top=objectives['DVH Objective'].str.contains('min', case=False) & objectives['Evaluator'].str.startswith('>')  & XMLdata.ObjType.isin(['top']) 
Mod10top=objectives['DVH Objective'].str.contains('max', case=False) & objectives['Evaluator'].str.startswith('<')  & XMLdata.ObjType.isin(['top']) 

Mod0bot=objectives['Evaluator'].str.startswith('>') & XMLdata.ObjType.isin(['bottom']) 
Mod1bot=objectives['Evaluator'].str.startswith('<') & XMLdata.ObjType.isin(['bottom']) 
Mod2bot=objectives['Evaluator'].str.startswith('=') & XMLdata.ObjType.isin(['bottom']) 

XMLdata.I3[Mod1top]=1
XMLdata.I3[Mod2top]=2
XMLdata.I3[Mod3top]=3
XMLdata.I3[Mod4top]=4
#XMLdata.I3[Mod5top]=5
#XMLdata.I3[Mod6top]=6
XMLdata.I3[Mod7top]=7
XMLdata.I3[Mod8top]=8
XMLdata.I3[Mod9top]=9
XMLdata.I3[Mod10top]=10

# XMLdata.I3[Mod0bot]=0
XMLdata.I3[Mod1bot]=1
XMLdata.I3[Mod2bot]=2


# Set the parameter values for the "top".

# Only need to set the parameter for mod 4 and 9, all the other ones are 0
XMLdata.I4[objectives['DVH Objective'].str.contains('min', case=False)]=100

#parameter for mod 0 and 1 - do we have this capability in our current excel sheet template?

# Set the "value" aka target values for the "bottom".  
for x in range(0,len(objectives),1):

    if bottom[x]:

        #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
        s=objectives.iloc[x,4]  

        #take out the [Gy],[cc],[%]
        stest=re.split('(\d+\D*\d*)',s)  #this notation means 1 or more digits followed by 0 or more special characters, 
                                         #followed by 0 or more digits

        if objectives.iloc[x,3].endswith('[cc]'):

        #split the first portion so you can extract the number
            XMLdata.I4[x]=str(float(stest[1])*1000) #convert cc to mm3

        else:
            XMLdata.I4[x]=str(float(stest[1]))

# Set the type specifier for the "bottom" and whether or not the units should be absolute or relative
for x in range(0,len(objectives),1):

    if Vobj[x] or Dobj[x]:


        #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
        t=objectives.iloc[x,3]  

        #take out the [Gy],[cc],[%]
        ttest=re.split('(\D+\[\D*\])',t)  #this notation means 1 or more non-numbers followed by [, followed by 0 or more non-numbers, followed by ]

        #split the first portion so you can extract the number
        ta=re.split('(\d+\D*\d*)',ttest[0]) #this means one or more digits followed by 0 or more non-numbers, followed by zero or more digits

        #convert the digit to a float to set the typespecifier for type 2,3,4,5
        XMLdata.I5[x]=str(ta[1]) 

        #check if the DQPvalue is in absolute units or not
        if re.search('\[%\]',ttest[1])==None:  
            XMLdata.I6[x]="true"
        else:
            XMLdata.I6[x]="false"


        if Type5[x]:
            XMLdata.I5[x]=str(float(XMLdata.I5[x]))  

    elif GIobj[x]|CIobj[x]:
        XMLdata.I5[x]="true"
        XMLdata.I6[x]="false"

#Set the dose (dose per fraction) and total dose for the "top" objectives

for x in range(0,len(objectives),1):

    if not bottom[x]:

        #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
        s=objectives.iloc[x,4]  

        #take out the [Gy],[cc],[%]
        stest=re.split('(\d+\D*\d*)',s)  #this notation means 1 or more digits followed by 0 or more special characters, followed by 0 or more digits

        #split the first portion so you can extract the number
        #calculate the dose per fraction


        # **************************************** THIS COULD BE AN ISSUE FOR SEQUENTIAL ******************************
        #  WHAT NUMBER OF FRACTIONS SHOULD BE USED?  SHOULD IT BE THE LARGEST??  THAT'S HOW IT'S CODED RIGHT NOW


        XMLdata.I5[x]=str(float(stest[1])/int(doseprescription['numfx'].iloc[-1])) #convert the digit to a float and divide it by number of fractions assumes Gy in clinical protocol

        #set the total dose value
        XMLdata.I6[x]=str(float(stest[1])) #convert the digit to a float set it as the TotalDose

# CSVdata = XMLdata.copy()

In [ ]:
# find the indeces of the rows that have dose constraints beyond treatment doses and delete them
# Find mean, min, and max dose constraints larger than maxdose
CSVdata = XMLdata.copy()
tempcsv = XMLdata.I6.copy()
objectives_edited = objectives.copy()

tempcsv[tempcsv.isin(['false'])]=0
tempcsv[tempcsv.isin(['true'])]=0

tempcsv0 = [float(i) for i in tempcsv] 
tempcsv0 = pd.Series(tempcsv0)
ind_delcsv0 = (tempcsv0>maxdose)

ind_to_delete_csv= CSVdata[(ind_delcsv0)].index
objectives_edited = objectives_edited.drop(ind_to_delete_csv)
# objectives_edited = objectives_edited.reset_index(drop=True) 

# find Dvol receiving Dose values where Dose>maxdose
tempcsv1 = [float(i) for i in CSVdata['I4']] 
tempcsv1 = pd.Series(tempcsv1)
ind_delcsv1=(CSVdata['I2']==5) & (CSVdata['I6']=='true') & (tempcsv1>maxdose)

# find VDoseGy values where Dose>maxdose
tempcsv2 = [float(i) for i in CSVdata['I5']] 
tempcsv2 = pd.Series(tempcsv2)
ind_delcsv2=(CSVdata['I2']==3) & (tempcsv2>maxdose)

ind_to_delete_csv= CSVdata[(ind_delcsv2 | ind_delcsv1)].index
objectives_edited = objectives_edited.drop(ind_to_delete_csv)
objectives_edited = objectives_edited.reset_index(drop=True) 

# Write the information into a new CSV file
CSVfolderpath = '//kodiak/physics/lpadilla/Clinic/10 week sprint - Peer review/test CSV files/'
CSVfilename = self.TxRegionFxType + ' - '+ finalfile + '.csv'
objectives_edited.to_csv(path_or_buf= CSVfolderpath + CSVfilename, index=False)

In [ ]:

# test = self.fullfilename.split(self.TxSite)
# test.replace(self.TemplateType + ' - ','')
# test[0] + finalfile
CSVfolderpath+CSVfilename 

In [ ]:
    av = 0  #index to keep track of how many acceptable variations there have been 


    for x in range(0,len(objectives),1):

        if not pd.isnull(objectives.Variation[x]):

#             print(str(x) + ' is not null:' + str(objectives.Variation[x])) #this indicates there is a variation for that objective


            if (type(objectives.Variation[x]) is not str) | str.isdigit(str(objectives.Variation[x])):# | objectives.Variation[x].replace('.','0').isdigit():        

                #this copies the data of the objective for which there is a variation so it can be inserted as a new objective in 
    #             the XML file
                variation = pd.DataFrame(XMLdata.loc[x+av,:]).transpose()



                if bottom[x]:

                    #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
                    s=objectives.Variation[x] 

                    variation.I4=str(s)

#                     print ('bottom variation for ' + str(x) + ':' + str (variation))

                    XMLdata = pd.concat([XMLdata.iloc[:x+1+av], variation, XMLdata.iloc[(x+1+av):]], ignore_index=True)

                    av = av + 1

                else:

                    #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
                    s=objectives.Variation[x]  

                    #split the first portion so you can extract the number
                    variation.I5=str(float(s)/numfractions[0]) #convert the digit to a float and divide it by number of fractions assumes Gy in clinical protocol
                    variation.I6=str(float(s)) #convert the digit to a float set it as the TotalDose  

#                     print ('top variation for ' + str(x) + ':' + str (variation))

                    XMLdata = pd.concat([XMLdata.iloc[:x+1+av], variation, XMLdata.iloc[(x+1+av):]], ignore_index=True)
                    av = av + 1

#             else: print('variation in next line!')
#         else: print('no acceptable variation')

     #CREATE THE PLAN OBJECTIVES XLM DATA FEED.  Conserve the order of the dose constraints
    XML_PO=XMLdata[XMLdata.ObjType.isin(['top'])].reset_index(drop=True) 
    XML_PO=XML_PO.append(XMLdata[XMLdata.ObjType.isin(['bottom'])]).reset_index(drop=True)    

In [ ]:
XMLdata

In [ ]:
#******************* Write out the data in XML format ********************************

#use the XML file below as the template to create all clinical protocols:
ClinicalProtocol = ET.parse('Clinical Protocol Template.xml')
tree=ClinicalProtocol
Protocol=ClinicalProtocol.getroot()
root=Protocol


# this just adds the correct extension to the file name for the clinical protocol
filename = finalfile +'.xml'


#these are the indexes that will ensure the objectives are written correctly on the XML file
idxt = 1  #index for top objectives
idxb = 1 #index for bottom Dose/Volume objectives
# idxb1 = 1 #index for bottom Dose/Volume objectives
# idxb2 = 1 #index for bottom CI/GI objectives


#insert the objectives following the correct format for each type

for x in range(0,len(XML_PO),1):
    data=XML_PO.loc[x,'I1':'I6'] #do it one objective at a time (one row of the dataframe per objective)
    
    if XML_PO.ObjType[x]=='top':
        convert_top(idxt,data,ClinicalProtocol,Protocol,filename) 
        idxt = idxt +1

        
    elif XML_PO.I2[x]>=2:  # all objectives of type 2 or larger are "bottom dose/volume" objectives
        convert_bottomDV(idxb,data,ClinicalProtocol,Protocol,filename) 
        idxb = idxb+1
        
    else:
        convert_bottomI(idxb,data,ClinicalProtocol,Protocol,filename)   
        idxb = idxb+1
        

        
# **************************************** THIS COULD BE AN ISSUE FOR SEQUENTIAL ******************************
#  WHAT NUMBER OF FRACTIONS SHOULD BE USED?  SHOULD IT BE THE LARGEST??  THAT'S HOW IT'S CODED RIGHT NOW
    
# set the number of fractions 
root.find('.//Phase//FractionCount').text = str(int(doseprescription['numfx'].iloc[-1]))  

# Change the clinical protocol name to the final file name so it shows the correct name when it's imported into Eclipse:
Protocol.find('.//Preview').set('ID', finalfile)
ClinicalProtocol.write(filename)


# ___________________________ END OF THE PLAN OBJECTIVES CSV TO XML CONVERSION_______________________________________
                


In [ ]:
#__________________________________ CREATE STRUCTURE TEMPLATE FOR CLINICAL PROTOCOL ___________________________________

#data info that will be used to create the XML structures:  
#
# 0 - structure name, 1 - type of structure, 2 - TypeIndex (always 2 for now), 3 - ColorAndStyle, 4 - DVHLineStyle (always 0 for now), 
# 5 - DVHLineColor (always -16777216), 6 - DVHLineWidth (always 1 for now)

#Create the XML matrix for the structure data
#Create the structure set based on the standardized list of organs suggested by PRO 2019 ASTRO consensus paper
# StructuresOfInterest=OrgansperTxSite[TxSite]

# or create the structure set based on the dose constraint sheet
StructuresOfInteresttemp=objectives['Structure ID'].unique()


# add any additional structures beyond what the dose constraint sheet lists

if 'Lung' in self.TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['Carina',
          'Liver',
          'Stomach',
          'Trachea'],
         )
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,OrgansperTxSite[TxSite])
    print('Lung')

elif ('CW' in self.TxSite) or ('Breast' in self.TxSite) :
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Larynx',
              'Liver',
              'Stomach',
              'Esophagus',
              'SpinalCord']
             )
    
    print('CW ' + ' Breast')
    
    
    if ('Breast L' in self.TxSite) :
        StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
                  ['Lumpectomy_L']
                 )  
        print('Breast L')
    
    if ('Breast R' in self.TxSite) :
        StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
                  ['Lumpectomy_R']
                 )  
        print('Breast R')
    

elif 'Liver' in self.TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Liver^Ex',
              'Liver^In',
              'Lungs']
             ) 
    print('Liver')
    
elif 'Pancreas' in self.TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Kidney_L',
              'Kidney_R',
              'Lungs']
             ) 
    print('Pancreas')
    
elif 'Esophagus' in self.TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Kidney_L',
              'Kidney_R',
              'Esophagus']
             ) 
    print('Esophagus')
    
elif ('Endometrium'  in self.TxSite) or ('Anus'  in self.TxSite) or ('Cervix' in self.TxSite) or ('Vulva or Vagina' in self.TxSite) :
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['BoneMarrow']
             ) 
    print('Endometrium/Anus/Cervix/Vulva/Vagina' )
    

elif 'Brain' in self.TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Brain',
#               'Glnd_Lacrimal',
#               'Hippocampus_L',
#               'Hippocampus_R',
#               'Pituitary',
#               'Scalp',
#               'A_Carotid',
              'OpticChiasm',
              'OpticNrv_L',
              'OpticNrv_R',
              'Lens_L',
              'Lens_R',
              'Eye_L',
              'Eye_R',
              'Cochlea_L',
              'Cochlea_R',
              'SpinalCord']             
             ) 
    print('Brain') 

elif 'Abdomen'  in self.TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Lungs']
             ) 
    print('Abdomen' )
    
    
# elif 'Cervix' in TxSite:
#     StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
#               ['BoneMarrow']
#              ) 
#     print('Endometrium/Anus/Cervix/Vulva/Vagina' )
    
# elif 'Vulva or Vagina' in TxSite:
#     StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
#               ['BoneMarrow']
#              ) 
#     print('Endometrium/Anus/Cervix/Vulva/Vagina' )
   
    
# delete duplicate structures
StructuresOfInterest=np.unique(StructuresOfInteresttemp)




In [ ]:
# Create a dataframe to capture all the relevant information for the structures listed for that treatment site
XML_S= pd.DataFrame(0,index=range(0,len(StructuresOfInterest)),columns=
                    ['StrID','Type','TypeIdx','Color','DVHLS','DVHLC','DVHLW','iCode','iCodeScheme','iCodeSchemeVersion'])

# Set the structure names based on the list of organs for that treatment site
XML_S.StrID=StructuresOfInterest 

#  Set all the default values
XML_S.TypeIdx=2   
XML_S.DVHLS=0    #Style
XML_S.DVHLC= -16777216  #Color
XML_S.DVHLW=1    #Width
XML_S.iCodeScheme='FMA'
XML_S.iCodeSchemeVersion='3.2'
# NOTE!!!!! the code scheme and version for fiducials does not come in correctly.  it stays with the default FMA but it should be 
# RADLEX 3.8

# find the colors that correspond to each organ using the OrganColors dictionary.  Organs that are derived like Lungs-ITV or 
# _PRV will have the same color as their original organs


XML_Stemp=XML_S 

        
av=0

for x in range (0,len(XML_Stemp),1):
    
    if XML_Stemp.StrID[x] in OrganColors: # if the structure is in the organ list, simply assign it the corresponding color
        XML_S.Color[x+av]=OrganColors[XML_Stemp.StrID[x]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[XML_Stemp.StrID[x]]
        
        if XML_Stemp.StrID[x].endswith('_L'):
            XML_S.DVHLS[x+av]=1
            
        
        
    elif '-' in XML_Stemp.StrID[x]: # if the structure is a derived structure from one found in the organ list,
                                #simply assign it the corresponding color
        temp=XML_Stemp.StrID[x].split('-') 
        
        XML_S.Color[x+av]=OrganColors[temp[0]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[temp[0]]
        XML_S.DVHLS[x+av]=2
        
        
        
        trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
        trueStr.StrID=temp[0]
        trueStr.Color=OrganColors[temp[0]]
        trueStr.Type='Organ'        
        trueStr.iCode=OrganCodes[temp[0]]
        
        XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
        av = av + 1
        
        
    elif '_PRV' in XML_Stemp.StrID[x]: # if the structure is a derived structure split it first,                                 
        
        temp=XML_Stemp.StrID[x].split('_PRV') 
        print(temp)
        
#         if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color 
        if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color             
            XML_S.Color[x+av]=OrganColors[temp[0]]
            XML_S.Type[x+av]='Organ'
            XML_S.iCode[x+av]='PRV'
            XML_S.DVHLS[x+av]=2
            
            trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
            
            trueStr.StrID=temp[0]
            trueStr.Color=OrganColors[temp[0]]
            trueStr.Type='Organ'        
            trueStr.iCode=OrganCodes[temp[0]] 
            trueStr.DVHLS[x+av]=0
        
            XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
            av = av + 1
            
    elif 'TV' in XML_Stemp.StrID[x]:
        XML_S.Type[x+av]='Target'
    
    else:
        print (StructuresOfInterest[x] + ' not found')
        
# Drop the duplicate structures      
if len(XML_S.StrID) != len(set(XML_S.StrID)):
    XML_S=XML_S.drop_duplicates(subset='StrID')  
    XML_S=XML_S.reset_index(drop=True) 

    
TargetColors=['Segment - Red','Segment - Orange','Segment - Pink', 'RGB255200150'] #set standard PTV colors

targets=XML_S[XML_S.Type.isin(['Target'])]  #find the structures set as "target" in the structure set from the dose constraints

ignorePorC=targets  #fill in a temporary array

for x in range (0,len(targets),1):
    ignorePorC.StrID[targets.index[x]]=targets.StrID[targets.index[x]][1:]  #take out the first letter of the target structure to give a given PTV, CTV, GTV
                                              #the same color
    
TargetVolumes=ignorePorC.StrID.unique()      #figure out how many unique targets we have

for x in range (0,len(TargetVolumes),1):
    XML_S.Color[XML_S['StrID'].str.contains(TargetVolumes[x])]=TargetColors[x]     #assigning them the specific colors
    
avt=0

#create GTV or CTV if there are none.

for x in range (0,len(TargetVolumes),1):
    
    avt=0

        
    if ('G'+TargetVolumes[x] not in XML_S['StrID']) and ('CW' not in self.TxSite) and ('Breast' not in self.TxSite) :    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)
        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()

        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='G' + temp[0]
        else:
            TVdef.StrID='G' + TargetVolumes[x]

        
        
        XML_S = pd.concat([XML_S.iloc[:idxTV+1+avt], TVdef, XML_S.iloc[(idxTV+1+avt):]], ignore_index=True)
        
        avt=1+avt

    if 'C'+TargetVolumes[x] not in XML_S['StrID']:    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='C' + temp[0]
#             TVdef.StrID='P' + temp[0]
        else:
            TVdef.StrID='C' + TargetVolumes[x]            
        
        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        
        avt=1+avt
        
        
    if TargetVolumes[x].endswith('_Eval'):    #create a PTV volume without the _Eval
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        temp=TargetVolumes[x].split('_Eval')
        TVdef.StrID='P' + temp[0]           
        
        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        
        avt=1+avt
        
        

XML_S.Type[XML_S.StrID.str.startswith('PTV')]='PTV'
XML_S.iCode[XML_S.StrID.str.startswith('PTV')]='PTVp'


XML_S.Type[XML_S.StrID.str.startswith('ITV')]='CTV' #no ITV available at the moment
XML_S.iCode[XML_S.StrID.str.startswith('ITV')]='ITV'


XML_S.Type[XML_S.StrID.str.startswith('CTV')]='CTV'
XML_S.iCode[XML_S.StrID.str.startswith('CTV')]='CTVp'


XML_S.Type[XML_S.StrID.str.startswith('GTV')]='GTV'
XML_S.iCode[XML_S.StrID.str.startswith('GTV')]='GTVp'
XML_S.iCode[XML_S.StrID.str.startswith('GTVn')]='GTVn'
XML_S.DVHLS[XML_S.StrID.str.startswith('GTV')]=3


for x in range(0,len(XML_S),1):
    
    if not str.isdigit(str(XML_S.iCode [x])):
        XML_S.iCodeScheme[x]='99VMS_STRUCTCODE'
        XML_S.iCodeSchemeVersion[x]='1.0'
        
XML_S=XML_S.drop_duplicates(subset='StrID').reset_index(drop=True)  

In [ ]:
# This will create the structures in the XML file of the clinical protocol
idxs = 1

for x in range(0,len(XML_S),1):
    
    data=XML_S.loc[x,'StrID':'iCodeSchemeVersion'] #One structure per row of the dataframe
    
    if XML_S.Type[x] != 0: 
        
        structure(idxs,data,tree,root,filename)
        idxs= idxs + 1
        print('success!!! ' + str(x) + ':' + data[0] )  
        
    else: 
        print ('Skipped: ' + XML_S.StrID[x])  
         
        
        
# Save the clinical protocol XML file in a specific folder
folderpath='L://Clinic/10 week sprint - Peer review/Completed clinical protocols for Eclipse/'
ClinicalProtocol.write(folderpath + filename)


# ___________________________ END OF THE STRUCTURE TEMPLATE CREATION IN XML FILE_______________________________________

# #create a log file entry
# ts = time.time()
# sttime = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H:%M:%S - ')
# with open(filepath+'z-log.csv', "a") as logfile:
#     logfile.write(sttime + ',' + csvfile + '\n')
    
    

In [ ]:


# Create a dataframe to capture all the relevant information for the structures listed for that treatment site
XML_S= pd.DataFrame(0,index=range(0,len(StructuresOfInterest)),columns=
                    ['StrID','Type','TypeIdx','Color','DVHLS','DVHLC','DVHLW','iCode','iCodeScheme','iCodeSchemeVersion'])

# Set the structure names based on the list of organs for that treatment site
XML_S.StrID=StructuresOfInterest 

#  Set all the default values
XML_S.TypeIdx=2   
XML_S.DVHLS=0    #Style
XML_S.DVHLC= -16777216  #Color
XML_S.DVHLW=1    #Width
XML_S.iCodeScheme='FMA'
XML_S.iCodeSchemeVersion='3.2'

# find the colors that correspond to each organ using the OrganColors dictionary.  Organs that are derived like Lungs-ITV or 
# _PRV will have the same color as their original organs


XML_Stemp=XML_S 

        
av=0

for x in range (0,len(XML_Stemp),1):
    
    if XML_Stemp.StrID[x] in OrganColors: # if the structure is in the organ list, simply assign it the corresponding color
        XML_S.Color[x+av]=OrganColors[XML_Stemp.StrID[x]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[XML_Stemp.StrID[x]]
        
        
    elif '-' in XML_Stemp.StrID[x]: # if the structure is a derived structure from one found in the organ list,
                                #simply assign it the corresponding color
        temp=XML_Stemp.StrID[x].split('-') 
        
        XML_S.Color[x+av]=OrganColors[temp[0]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[temp[0]]
        
        
        
        trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
        trueStr.StrID=temp[0]
        trueStr.Color=OrganColors[temp[0]]
        trueStr.Type='Organ'        
        trueStr.iCode=OrganCodes[temp[0]]
        
        XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
        av = av + 1
        
        
    elif '_PRV' in XML_Stemp.StrID[x]: # if the structure is a derived structure split it first,                                 
        
        temp=XML_Stemp.StrID[x].split('_PRV') 
        print(temp)
        
#         if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color 
        if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color             
            XML_S.Color[x+av]=OrganColors[temp[0]]
            XML_S.Type[x+av]='Organ'
            XML_S.iCode[x+av]='PRV'
            
            trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
            
            trueStr.StrID=temp[0]
            trueStr.Color=OrganColors[temp[0]]
            trueStr.Type='Organ'        
            trueStr.iCode=OrganCodes[temp[0]] 
        
            XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
            av = av + 1
            
    elif 'TV' in XML_Stemp.StrID[x]:
        XML_S.Type[x+av]='Target'
    
    else:
        print (StructuresOfInterest[x] + ' not found')
        
# Drop the duplicate structures      
if len(XML_S.StrID) != len(set(XML_S.StrID)):
    XML_S=XML_S.drop_duplicates(subset='StrID')  
    XML_S=XML_S.reset_index(drop=True) 

    
TargetColors=['Segment - Red','Segment - Orange','Segment - Pink', 'RGB255200150'] #set standard PTV colors

targets=XML_S[XML_S.Type.isin(['Target'])]  #find the structures set as "target" in the structure set from the dose constraints

ignorePorC=targets  #fill in a temporary array

# for x in range (0,len(targets),1):
#     ignorePorC.StrID[x]=targets.StrID[x][1:]  #take out the first letter of the target structure to give a given PTV, CTV, GTV
#                                               #the same color
    
# TargetVolumes=ignorePorC.StrID.unique()      #figure out how many unique targets we have

for x in range (0,len(targets),1):
    ignorePorC.StrID[targets.index[x]]=targets.StrID[targets.index[x]][1:]  #take out the first letter of the target structure to give a given PTV, CTV, GTV
                                              #the same color
    
TargetVolumes=ignorePorC.StrID.unique()      #figure out how many unique targets we have

if len(doseprescription)>1:
#     TargetVolumes=sorted(TargetVolumes,reverse=True)
    TargetVolumes=sorted(TargetVolumes,reverse=False)
    
for x in range (0,len(TargetVolumes),1):
    XML_S.Color[XML_S['StrID'].str.contains(TargetVolumes[x])]=TargetColors[x]     #assigning them the specific colors
    
avt=0

#create GTV or CTV if there are none.

for x in range (0,len(TargetVolumes),1):
    
    avt=0

        
    if ('G'+TargetVolumes[x] not in XML_S['StrID']) and ('CW' not in self.TxSite) and ('Breast' not in self.TxSite) :    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)
        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()

        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='G' + temp[0]
        else:
            TVdef.StrID='G' + TargetVolumes[x]
      
        

        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        avt=1+avt

    if 'C'+TargetVolumes[x] not in XML_S['StrID']:    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='C' + temp[0]
#             TVdef.StrID='P' + temp[0]
        else:
            TVdef.StrID='C' + TargetVolumes[x]            
        

        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        avt=1+avt
        
        
    if TargetVolumes[x].endswith('_Eval'):    #create a PTV volume without the _Eval
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        temp=TargetVolumes[x].split('_Eval')
        TVdef.StrID='P' + temp[0]   
        

        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        avt=1+avt
        
        

XML_S.Type[XML_S.StrID.str.startswith('PTV')]='PTV'
XML_S.iCode[XML_S.StrID.str.startswith('PTV')]='PTVp'


XML_S.Type[XML_S.StrID.str.startswith('ITV')]='CTV' #no ITV available at the moment
XML_S.iCode[XML_S.StrID.str.startswith('ITV')]='ITV'


XML_S.Type[XML_S.StrID.str.startswith('CTV')]='CTV'
XML_S.iCode[XML_S.StrID.str.startswith('CTV')]='CTVp'


XML_S.Type[XML_S.StrID.str.startswith('GTV')]='GTV'
XML_S.iCode[XML_S.StrID.str.startswith('GTV')]='GTVp'
XML_S.iCode[XML_S.StrID.str.startswith('GTVn')]='GTVn'

for x in range(0,len(XML_S),1):
    
    if not str.isdigit(str(XML_S.iCode [x])):
        XML_S.iCodeScheme[x]='99VMS_STRUCTCODE'
        XML_S.iCodeSchemeVersion[x]='1.0'

XML_S=XML_S.drop_duplicates(subset='StrID').reset_index(drop=True)  


In [ ]:
###-------Chose A CSV using a dialogue rather than manually above-------------------------------------------------------

#--Libraries needed--
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

#--don't know much, but these two lines get rid of an anoying little box that would stay after finishing---
root = tk.Tk()
root.withdraw()

#--Select the CSV file from the "DC Sheets" directory---
fullfilename = filedialog.askopenfilename(initialdir = filepath, title = "Select CSV file", filetypes = (("CSV files","*.csv"),("all files","*.*")))

#-- "fullfilename" is now the whole path. Next, remove the path to leave just the file name
filename = fullfilename.replace(filepath,"") # To impliment in your code, replace "filename" with "csvfile"

###-----------------------------------------------------------------------------------------------------------------------


# Move this to the end of the program if you want just to indicate that the program is finshed
# Change the message to whatever.
finished = messagebox.showinfo("Create XML","End of program message")


#--oh and maybe we'd want to conclude by opening the directory with the xml file?
import webbrowser
webbrowser.open('file:///' + '//kodiak/physics/lpadilla/Clinic/10 week sprint - Peer review/Completed clinical protocols for Eclipse/')


In [ ]:
A = np.random.randn(4,3)
B = np.sum(A, axis = 1, keepdims = True)

In [ ]:
B.shape

In [ ]:
#create a log file entry
ts = time.time()
sttime = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H:%M:%S - ')
with open(filepath+'z-log.csv', "a") as logfile:
    logfile.write(sttime + ',' + csvfile + '\n')


In [ ]:
# THIS CELL IS FOR WHEN THE STRUCTURE SET IS LOADED FROM THE PRO DATA, NOT THE DOSE CONSTRAINTS

# # Create a dataframe to capture all the relevant information for the structures listed for that treatment site
# XML_S= pd.DataFrame(0,index=range(0,len(StructuresOfInterest)),columns=
#                     ['StrID','Type','TypeIdx','Color','DVHLS','DVHLC','DVHLW'])

# # Set the structure names based on the list of organs for that treatment site
# XML_S.StrID=StructuresOfInterest 

# # Since all the structures listed in this dictionary are organs, set the type for all of them to "Organ"
# XML_S.Type='Organ' 

# #  Set all the default values
# XML_S.TypeIdx=2   
# XML_S.DVHLS=0
# XML_S.DVHLC= -16777216
# XML_S.DVHLW=1

# # find the colors that correspond to each organ using the OrganColors dictionary
# for x in range (0,len(XML_S),1):
#     XML_S.Color[x]=OrganColors[StructuresOfInterest[x]]
    
# # Make sure that all the organs for that tx site have a color assigned already.  If they don't, go into the OrganColors 
# # dictionary and assign one

# XML_S    

In [ ]:
# This creates the blank clinical protocol template.  YOU DO NOT NEED TO RUN THIS UNLESS SOMETHING HAPPENS TO THE 
# TEMPLATE AND IT NEEDS TO BE REDONE

ClinicalProtocol = ET.parse('Clinical protocol test.xml')
tree=ClinicalProtocol
Protocol=ClinicalProtocol.getroot()
root=Protocol
filename='Clinical Protocol Template.xml'

for child in Protocol.find('.//Prescription'): 
    Protocol.find('.//Prescription').remove(child)
    ClinicalProtocol.write(filename)

Protocol.find('.//Structures').clear()
Protocol.find('.//Phases//Phase//FractionCount').clear()
root.find('.//Phases//Phase//FractionCount').text ='25'
Protocol.find('.//Phases//PlanTemplate//FractionCount').clear()
root.find('.//Phases//PlanTemplate//FractionCount').text ='25'

Protocol.find('.//Preview').set('ID', 'Clinical Protocol Template')
ClinicalProtocol.write(filename)
